<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%run Packages/imports_buy.py
%run Stocks/Combine_Stock.py

<br> Combine Stocks and calculate the values for each given indicator </br>

In [4]:
stocks_dax = ['ADS.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'CON.DE', '1COV.DE', 'DAI.DE', 'DHER.DE', 'DBK.DE', 'DB1.DE', 'DPW.DE', 'DTE.DE', 'DWNI.DE', 'EOAN.DE', 'FRE.DE', 'FMS', 'HEI.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 'MRK', 'MTX.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'ENR.F', 'VOW.DE', 'VNA.DE']
stocks_random = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'V', 'YNDX']

stock_list_1 = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE']
stock_list_2 = ['PLUG', 'MDO.DE', 'PYPL', 'V', 'YNDX']

#stocks_combine = np.concatenate([stocks_dax, stocks_random])
df_complete = combine_stocks_today(stock_list_1)

In [5]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

<br> Calculate a score based on the indicators </br>

In [8]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

def macd_signalline_score(row): 
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

# set weighting when close is lower lower_band or upper upper_band
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>=middle_band):
        if adj_close>=upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<=lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = (lower_band + upper_band) / 2 
    middle_band1 = 55
    middle_band2 = 45
    outlier_factor = 1.2
    rsi = row['RSI']
    if (rsi>middle_band):
        if rsi>=upper_band:
            score = ((rsi/upper_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                if rsi<=62.5:
                    score = rsi/middle_band1-1
                else:
                    score = ((62.5-(rsi-62.5))/(middle_band1))-1
    else:
        if (rsi<=lower_band):
            score = ((rsi/lower_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi>middle_band2:
                score = 0
            else:
                if rsi>=37.5:
                    score = (rsi/middle_band2-1)
                else:
                    score = (37.5-(rsi-37.5))/middle_band2-1
    return score

# We should check whether we give the same weighting if it hits above or lower 100/-100
def cci_score(row):
    lower_band = -100
    middle_lower_band = -50
    upper_band = 105
    middle_band = 0
    middle_upper_band = 85
    outlier_factor = 0.25
    outlier_factor1 = 2
    cci=row['RSI']

    if (cci < lower_band):
        score=(cci/lower_band)-1
    else:
        if (cci<=middle_lower_band):
            score = (cci / lower_band)-1
        elif (cci<middle_band):
            score = (((cci-lower_band)/lower_band)+1)*(-1)
        else:
            if (cci>upper_band):
                score = ((cci/upper_band)-1)*(-1)
            else:
                if(cci<=middle_upper_band):
                    score = (cci/upper_band)*outlier_factor
                else:
                    if(cci<upper_band):
                        score=(middle_upper_band-(cci-middle_upper_band))/upper_band-(middle_upper_band-(middle_upper_band-upper_band))/middle_upper_band
                    else:
                        score=0
    
    return score

## TO BE ADDED: DEAD-CAT-BOUNCE SCORE CALC  

def relative_score(row):
    r = row
    stock = row['stock_id']

    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)
    cci_scr = cci_score(r)

    score = ema_scr + (macd_signalline_scr*5) + bollinger_scr + sma_scr + rsi_scr + cci_scr
    print(stock, ema_scr, (macd_signalline_scr*5), bollinger_scr, sma_scr, rsi_scr, cci_scr, score, '\n')
    return score

In [9]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.05443898293852678 -0.013649102479347253 0.018375179848447276 0.011849355990963817 0 0.10734912382812334 0.17836354012671396 

AAPL 0.06877263809991652 -0.015764441988677103 0.018841239218665984 0.016769900581251163 0 0.1212999806304756 0.20991931654163215 

SAP 0.03226353864906173 -0.025316721351720806 0.030014305060250046 -0.0027009456099866824 0 0.11226567372813247 0.14652585047573674 

ACN 0.07016897365039432 -0.0010922036157423598 0.002194327344490543 0.030270305426058908 0.13052166839336254 0.1495745434246787 0.38163761462324264 

VOW.DE 0.05364383100080716 0.012005142627027343 0.013681878718789786 0.01011135261317131 0.039793209448618194 0.13616339647541428 0.2653988108838281 



<ipython-input-9-fd477b20e8aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)


,Open,High,Low,Close,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
date,,,,,,,,,,,,,,,,,,,
2021-07-28,320.85,321.0000,316.1200,316.31,316.31,ACN,307.746630,287.568260,307.016516,286.762605,310.055636,297.768850,323.955592,6.765045,6.834140,62.821308,80.215394,False,0.381638
2021-07-28,279.60,280.4000,274.6000,278.20,278.20,VOW.DE,277.177367,263.065524,275.415180,276.966635,276.927301,263.455634,287.190896,-0.480778,-1.148744,57.188627,-21.677319,False,0.265399
2021-07-28,144.81,146.9700,142.5400,144.98,144.98,AAPL,142.594389,133.418825,142.588800,130.965489,143.829906,137.640791,151.698209,3.394801,3.851906,50.945992,-54.631497,False,0.209919
2021-07-28,3633.78,3658.4204,3601.0000,3630.32,3630.32,AMZN,3578.923136,3394.149110,3587.806800,3337.660100,3598.813147,3447.138156,3797.739844,51.034616,60.944738,45.086632,-12.086933,False,0.178364
2021-07-28,141.66,143.3300,141.2179,142.97,142.97,SAP,142.782745,138.320050,143.357200,136.300622,142.822502,135.967649,151.965351,-0.341163,0.382743,47.151583,-26.310220,False,0.146526
